# Human Responses

In [1]:
import pandas as pd
import numpy as np
import random

np.random.seed(0)
random.seed(0)

In [2]:
# We use min(8, 9) = 8 prompts, so both corpuses have the same number of prompts
n_prompts = 8
# We use the longest n_samples_per_prompt samples for each prompt, measured by number of words
n_samples_per_prompt = 100

In [3]:
index_0 = np.concatenate([["prompt_id", "text"], ["bert"] * 768])
index_1 = np.concatenate([["prompt_id", "text"], [i for i in range(768)]])
index = pd.MultiIndex.from_arrays([index_0, index_1])

## Reddit

In [4]:
pairs = []
for name in ["train", "test", "valid"]:
    with open("reddit/writingPrompts/" + name + ".wp_source") as f_prompts:
        prompts = f_prompts.readlines()
    with open("reddit/writingPrompts/" + name + ".wp_target") as f_responses:
        texts = f_responses.readlines()
    assert len(prompts) == len(texts)
    pairs.extend(list(zip(prompts, texts)))
df_reddit = pd.DataFrame(pairs, columns=["prompt", "text"])
df_reddit = df_reddit[~df_reddit["prompt"].str.contains("hitler", case=False)]
df_reddit = df_reddit.drop_duplicates(subset=["text"])
df_reddit = df_reddit[
    df_reddit["prompt"].isin(df_reddit["prompt"].value_counts().index[:n_prompts])
]
df_reddit["prompt_id"] = df_reddit["prompt"].astype("category").cat.codes

# Get the longest n_samples responses for each prompt
df_reddit["text_len"] = df_reddit["text"].apply(lambda x: len(str.split(x)))
df_reddit = (
    df_reddit.groupby("prompt_id")
    .apply(lambda x: x.nlargest(n_samples_per_prompt, "text_len"), include_groups=False)
    .reset_index(level=0, drop=False)
    .reset_index(drop=True)
)

# Break prompt into prompt and and prompt_tag
df_reddit["prompt_tag"] = (
    df_reddit["prompt"]
    .str.split(" \]", n=1)
    .str[0]
    .replace("\[", "", regex=True)
    .str.strip()
)
df_reddit["prompt"] = df_reddit["prompt"].str.split(" \]", n=1).str[1].str.strip()
df_reddit["prompt"] = "Prompt\n" + df_reddit["prompt"]
df_reddit["prompt_id"] = df_reddit["prompt"].astype("category").cat.codes

In [5]:
df_reddit_prompts = (
    df_reddit[["prompt_id", "prompt", "prompt_tag"]]
    .drop_duplicates()
    .sort_values("prompt_id")
    .reset_index(drop=True)
)
df_reddit_prompts.to_csv("reddit_prompts.csv", index=False)

In [6]:
df_reddit = df_reddit[["prompt_id", "text"]]
df_reddit = df_reddit.sort_values("prompt_id").reset_index(drop=True)
df_reddit.to_csv("reddit.csv", index=False)

```
WP: Writing Prompt
SP: Simple Prompt
EU: Established Universe
CW: Constrained Writing
TT: Theme Thursday
PM: Prompt Me
MP: Media Prompt
IP: Image Prompt
PI: Prompt Inspired
OT: Off Topic
* OT as an Advertisement!
RF: Reality Fiction
```

https://www.reddit.com/r/WritingPrompts/wiki/how_to_tag_prompts/

In [7]:
df_reddit_prompts

,prompt_id,prompt,prompt_tag
0,0,Prompt\nA peaceful alien race is besieged by a...,WP
1,1,Prompt\nThere is no prompt . Just write a stor...,WP
2,2,Prompt\nThis is the prologue ( or the first ch...,WP
3,3,Prompt\nWrite a short story where the first se...,WP
4,4,Prompt\nWrite the first and last paragraph of ...,CW
5,5,Prompt\nWrite the letter that you always wante...,WP
6,6,Prompt\nYou live in a city full of people with...,WP
7,7,Prompt\n`` She said she loved him . '' Insert ...,WP


In [8]:
df_reddit

,prompt_id,text
0,0,`` We left them there to study ! '' Proclaimed...
1,0,“ I suggest we initiate protocol Zestraol ” <n...
2,0,Our War Council was surprised when these Human...
3,0,"`` Drax , the Slovians have taken E13-49e , 4t..."
4,0,"`` They 've taken Marin , sir . '' <newline> <..."
...,...,...
795,7,"I 'd see her walking down the hall , her hair ..."
796,7,Wait . <newline> <newline> Doubt was settling ...
797,7,It started as a chauvinistic affair meant to m...
798,7,Izzard stalked through the once slicked stone ...


In [9]:
print("Number of words in the responses:")
df_reddit["text"].apply(lambda x: len(x.split())).describe()

Number of words in the responses:


count     800.00000
mean      695.49625
std       514.91980
min       121.00000
25%       290.00000
50%       520.00000
75%       932.25000
max      2594.00000
Name: text, dtype: float64

## Hewlett

https://www.kaggle.com/competitions/asap-aes/code

In [10]:
import os

hewlett_prompts_dir = "hewlett/prompts"

prompts = []
for file in os.listdir(hewlett_prompts_dir):
    with open(hewlett_prompts_dir + "/" + file) as f:
        prompt = f.read()
    prompts.append((int(file.split(".")[0]) - 1, prompt))

df_hewlett_prompts = pd.DataFrame(prompts, columns=["prompt_id", "prompt"])
df_hewlett_prompts["prompt_tag"] = df_hewlett_prompts["prompt"].str.contains(
    "Source Essay"
)
df_hewlett_prompts["prompt_tag"] = df_hewlett_prompts["prompt_tag"].replace(
    {True: "source dependent responses", False: "persuasive / narrative / expository"}
)
df_hewlett_prompts = df_hewlett_prompts.sort_values("prompt_id").reset_index(drop=True)
df_hewlett_prompts.to_csv("hewlett_prompts.csv", index=False)

In [11]:
hewlett_dir = "hewlett"

filenames = [
    "training_set_rel3.tsv",
    "valid_set.tsv",
    "test_set.tsv",
]

dfs = []
for filename in filenames:
    df = pd.read_csv(f"{hewlett_dir}/{filename}", sep="\t", encoding="ISO-8859-1")
    df = df[["essay_set", "essay"]]
    df.rename(columns={"essay_set": "prompt_id", "essay": "text"}, inplace=True)
    df["prompt_id"] = df["prompt_id"].astype(int).apply(lambda x: x - 1)
    dfs.append(df)

# Don't need to remove the responses of any prompts because there are 8 distinct prompts in this dataset
df_hewlett = pd.concat(dfs, ignore_index=True)

df_hewlett = df_hewlett[df_hewlett["text"] != ""]
df_hewlett = df_hewlett.dropna()
df_hewlett = df_hewlett.drop_duplicates()

# Get the longest n_samples responses for each prompt
df_hewlett["text_len"] = df_hewlett["text"].apply(lambda x: len(str.split(x)))
df_hewlett = (
    df_hewlett.groupby("prompt_id")
    .apply(lambda x: x.nlargest(n_samples_per_prompt, "text_len"), include_groups=False)
    .reset_index(level=0, drop=False)
    .reset_index(drop=True)
)
df_hewlett = df_hewlett[["prompt_id", "text"]]

df_hewlett.to_csv("hewlett.csv", index=False)

In [12]:
df_hewlett_prompts

,prompt_id,prompt,prompt_tag
0,0,"Prompt\nMore and more people use computers, bu...",persuasive / narrative / expository
1,1,"Prompt\nCensorship in the Libraries\n""All of u...",persuasive / narrative / expository
2,2,Source Essay\nROUGH ROAD AHEAD: Do Not Exceed ...,source dependent responses
3,3,Source Essay\nWinter Hibiscus by Minfong Ho\nS...,source dependent responses
4,4,Source Essay\nNarciso Rodriguez\nfrom Home: Th...,source dependent responses
5,5,Source Essay\nThe Mooring Mast\nby Marcia Amid...,source dependent responses
6,6,Prompt\nWrite about patience. Being patient me...,persuasive / narrative / expository
7,7,Prompt\nWe all understand the benefits of laug...,persuasive / narrative / expository


In [13]:
df_hewlett

,prompt_id,text
0,0,My standing postion on this cause is that comp...
1,0,"@ORGANIZATION1, @CAPS1? Are you there?"" ""@CAPS..."
2,0,"Dear The @CAPS1 newspaper, @CAPS2 in front of ..."
3,0,Dear @CAPS1 Society: Computers are perhaps one...
4,0,"Dear @ORGANIZATION1, The creation of computers..."
...,...,...
795,7,"We couldn't control our selves, our eyes wate..."
796,7,It all started at the play ground @CAPS9 me ...
797,7,For my family laughter is important to us bec...
798,7,"Laughter, one of the greatest gifts in life. ..."


In [14]:
print("Number of words in the responses:")
df_hewlett["text"].apply(lambda x: len(x.split())).describe()

Number of words in the responses:


count     800.000000
mean      465.585000
std       246.627283
min       205.000000
25%       254.000000
50%       351.500000
75%       718.000000
max      1064.000000
Name: text, dtype: float64

In [15]:
"""
# Expand the bert column into 768 columns
df_hewlett = pd.concat(
    [df_hewlett, pd.DataFrame(np.zeros((len(df_hewlett), 768)))], axis=1
)
df_hewlett.columns = index
"""

'\n# Expand the bert column into 768 columns\ndf_hewlett = pd.concat(\n    [df_hewlett, pd.DataFrame(np.zeros((len(df_hewlett), 768)))], axis=1\n)\ndf_hewlett.columns = index\n'

In [16]:
x = df_hewlett[df_hewlett["prompt_id"] == 7].head(3)["text"].to_list()

for y in x:
    print(y)
    print()

  Laughter in life is a good thing to have I believe. Once @CAPS3 I was @NUM1 year's old I had a and @CAPS1. We always loved to hang out and do things together like take walks, or go to the beach, or go to the parks. What was so funny was that that we also loved to eat together too, maybe not out his dog bowl, but off of plates or on a picnic table. He was a cute dog. He had four little puppies named @CAPS5-@CAPS6, popcorn,big tail, and mountain. I named them all weird animal names because that's either what they smelled like or that's what they looked or loved doing.  Why we named my dog @CAPS1 was because he always smelt like @CAPS1. I always took him on walks to make him a healthy dog. It made me very angry @CAPS3 he had puppies and a mate. I don't know why maybe just because head I really loved my dog and i wanted him to be mine only. My favorite friend. I thought in my head hey! that's my dog and you leave him alone and do not touch him. Well @CAPS3 he had puppies i finally realiz

# LLM response generation

In [17]:
def modify_prompt(prompt, dataset):
    if dataset == "reddit":
        prompt = prompt.split("\n")[1]
        return (
            "Write a response the preceding creative writing prompt:\n" + prompt + "\n"
        )
    else:
        return prompt + "\n"

## Gemini (1.0 and 1.5)

In [18]:
import pandas as pd
import time
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold


with open("API_KEY_GOOGLE.txt", "r") as f:
    API_KEY_GOOGLE = f.read()

genai.configure(api_key=API_KEY_GOOGLE)


safety_settings = {
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
}

gemini_10_pro = genai.GenerativeModel(
    "models/gemini-1.0-pro", safety_settings=safety_settings
)

gemini_15_pro = genai.GenerativeModel(
    "models/gemini-1.5-pro-latest", safety_settings=safety_settings
)


def generate_gemini(model, model_name, prompt):
    while True:
        start = time.time()
        response = model.generate_content(prompt)
        # Gemini sometimes returns an empty response due to "SAFETY", so try again
        if not response.parts:
            print(response.candidates)
            continue
        # Gemini has a rate limit of 15 requests per minute for 1.0 and 2 requests per minute for 1.5
        wait_time = 30 if "1.5" in model_name else 4
        time.sleep(max(0, wait_time + 1 - (time.time() - start)))
        return " ".join([part.text for part in response.parts])

/home2/hstropkay/llm-style/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## GPT (3.5 and 4.0 and 3.5 over a range of temperature values)

In [ ]:
from openai import OpenAI

with open("API_KEY_OPENAI.txt", "r") as f:
    API_KEY_OPENAI = f.read()

client = OpenAI(api_key=API_KEY_OPENAI)

model_35 = "gpt-3.5-turbo-0125"
model_4 = "gpt-4-turbo-2024-04-09"

completion = client.chat.completions.create(
    model=model_35,
    messages=[
        {
            "role": "system",
            "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair.",
        },
        {
            "role": "user",
            "content": "Compose a poem that explains the concept of recursion in programming.",
        },
    ],
)

print(completion.choices[0].message)

def generate_gpt(model,

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
from tqdm import tqdm

r = []
df_hewlett_prompts = pd.read_csv("hewlett_prompts.csv")
prompts = df_hewlett_prompts["prompt"].to_list()
for prompt in tqdm(prompts):
    prompt = modify_prompt(prompt, "hewlett")
    r.append(generate_gemini(gemini_15_pro, "gemini-1.5-pro", prompt))
r

In [ ]:
"""
threads = []
responses = np.zeros(len(prompts_800), dtype=object)
for i, prompt in enumerate(tqdm(prompts_800)):
    thread = threading.Thread(target=generate_response, args=(prompt, responses, i))
    thread.start()
    threads.append(thread)
    time.sleep(2)

for thread in threads:
    thread.join()

responses = list(responses)
df = pd.DataFrame({"prompt": prompts_800, "response": responses})
df.to_csv("gemini1.csv", index=False)
"""